In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import spacy

#   spaCy model
def load_spacy_model():
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        from spacy.cli import download
        download("en_core_web_sm")
        return spacy.load("en_core_web_sm")

nlp = load_spacy_model()

app = Flask(__name__)
CORS(app)

# Dummy data (later replace with PostgreSQL)
NONPROFIT_OPPORTUNITIES = [
    {"id": 1, "title": "Beach Cleanup Crew Leader", "description": "Lead a team of volunteers for coastal cleanups. Requires strong organizational and communication skills. Interest in environmental conservation is a plus.", "organization": "Conservation Corps NL"},
    {"id": 2, "title": "Social Media Coordinator", "description": "Help manage our social media channels to raise awareness about youth mental health. Experience with platforms like Instagram and TikTok is needed.", "organization": "Choices for Youth"},
    {"id": 3, "title": "Kitchen Helper for Soup Kitchen", "description": "Assist with food preparation and serving at our community kitchen. A basic level of cooking skills is helpful.", "organization": "Stella's Circle"}
]

@app.route('/api/match', methods=['POST'])
def match_opportunities():
    try:
        data = request.json
        volunteer_skills = data.get('skills', '').strip()
        volunteer_interests = data.get('interests', '').strip()

        if not volunteer_skills and not volunteer_interests:
            return jsonify({"error": "Please provide skills or interests"}), 400

        volunteer_text = nlp(volunteer_skills + " " + volunteer_interests)

        matches = []
        for opp in NONPROFIT_OPPORTUNITIES:
            opp_doc = nlp(opp['description'])
            similarity = volunteer_text.similarity(opp_doc)
            matches.append({
                "opportunity": opp,
                "relevance_score": round(similarity, 2)
            })

        sorted_matches = sorted(matches, key=lambda x: x['relevance_score'], reverse=True)
        return jsonify(sorted_matches)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)
